In [4]:
# Standard libs for data manipulation
import numpy as np
import pandas as pd
import json
import io
import datetime
from datetime import date
from datetime import timedelta
import re
#---- DATABASE MANAGEMENT TOOLS --------------#
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras as extras

#---- OWN MODULE IMPORTS --------------------#
import config.pw
import ValuationModel
from ValuationModel.assist_functions import *
from ValuationModel.fmp import *
from config.api import MY_API_KEY

import warnings
warnings.filterwarnings('ignore')



In [5]:
#=============SET UP LOGGING ======================#
import logging
import sys
# specifically for pyplot: https://stackoverflow.com/questions/56618739/matplotlib-throws-warning-message-because-of-findfont-python
logging.getLogger('matplotlib').disabled = True

logger=logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(lineno)s - %(levelname)s - %(message)s')
logger.setLevel(logging.DEBUG)

# set logging handler in file
fileHandler=logging.FileHandler(filename="log/kpi.log", mode='w')
fileHandler.setFormatter(formatter)
fileHandler.setLevel(logging.DEBUG)
logger.addHandler(fileHandler)
#===================================================#

In [6]:
tickerdict=json.load(open('ticker_dict.json', 'r'))
global ticker_list
ticker_list=list(tickerdict.values())
#ticker_list=['GTLB', 'PLTR', 'IFX.DE', 'EOAN.DE', 'SHL.DE', 'PAH3', 'NEL.OL', 'PLUG', 'DIS', 'KO', 'MMM', 'MSFT', 'SQ', 'ABNB', 'AMZN', 'EBAY', 'ORCL', 'PYPL', 'QCOM', 'SNOW', 'TEAM', 'TSLA', 'DBK.DE', 'PM']

company='The Walt Disney Company' # Currently, I have just a limited amount of companies in the database. The companies are: GitLab Inc., INFINEON TECHNOLOGIES AG, E.ON SE, Palantir Technologies Inc.,

ticker=tickerdict[company]

In [7]:
# Set necessary url variables for the sqlalchemy create_engine() method.
user='svenst89' # or default user 'postgres'
password=config.pw.password # edit the password if you switch to the default user 'postgres'; I setup different passwords.
host='localhost'
port='5432'
database='fundamentalsdb'

In [8]:
global engine
def run_engine():
    return create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

In [9]:
#===OPTIONAL: INSTEAD OF USING FUNCTION 'get_database_findata_year()' use the general function retrieving all available financial data and extracting a specific year from the whole dataframe
engine = run_engine()
latest_year=get_latest_available_fy(ticker=ticker, json_entry="date")
bs, incs, cs = get_database_findata(company, engine)
year=2022 #get_latest_available_fy(ticker, json_entry="date")
bs_y=clearDataframes_and_get_fy(company, bs, 'bs', latest_year)
incs_y=clearDataframes_and_get_fy(company, incs, 'incs', latest_year)
cs_y=clearDataframes_and_get_fy(company, cs, 'cs', latest_year)

You have successfully retrieved the financial statement data for company 'The Walt Disney Company'!
==> I revised the Balance Sheet and extracted financial data for Fiscal Year (FY) 2022.

==> I revised the Income Statement and extracted financial data for Fiscal Year (FY) 2022.

==> I revised the Cashflow Statement and extracted financial data for Fiscal Year (FY) 2022.



In [10]:
bs

,id,shortname,date,item,value,company_id
0,5883,The Walt Disney Company,2022-10-01,calendarYear,2022,7
1,5884,The Walt Disney Company,2022-10-01,period,FY,7
2,5885,The Walt Disney Company,2022-10-01,symbol,DIS,7
3,5886,The Walt Disney Company,2022-10-01,reportedCurrency,USD,7
4,5887,The Walt Disney Company,2022-10-01,cik,0001744489,7
...,...,...,...,...,...,...
2009,7892,The Walt Disney Company,1985-09-30,totalInvestments,0,7
2010,7893,The Walt Disney Company,1985-09-30,totalDebt,823100000,7
2011,7894,The Walt Disney Company,1985-09-30,netDebt,823100000,7
2012,7895,The Walt Disney Company,1985-09-30,link,,7
